In [ ]:
#importing required libraries
import cv2
import imutils
import os
import numpy as np
import easygui
import imageio
import sys
import tkinter as tk
from tkinter import filedialog
from tkinter import *
import matplotlib.pyplot as plt
from PIL import ImageTk, Image

#entering the cluster k required and grey scale level
k= int(input("Enter the number of colors to be focused for clustering (k-means): "))
gScale= int(input("Enter the grey scale value: "))

#function for getting the path of real image location when button pressed
def uploadOriginal():
    pathOfImage=easygui.fileopenbox()
    cartoon(pathOfImage)


#function for cartoonification
def cartoon(pathOfImage):
    realImage = cv2.imread(pathOfImage)
    realImage = cv2.cvtColor(realImage, cv2.COLOR_BGR2RGB)

#analyse selected entity is image
    if realImage is None:
        print("ERROR: Please upload an image !")
        sys.exit()

#changing the size of image
    sizeChange = cv2.resize(realImage, (654, 874))

#greyscale conversion of real image
    imageGreyScaling = cv2.cvtColor(realImage, cv2.COLOR_BGR2GRAY)
    greyScaledSized = cv2.resize(imageGreyScaling, (654, 874))

#smoothening the image using median blur
    smootheningImage = cv2.medianBlur(imageGreyScaling, gScale)
    smoothSized = cv2.resize(smootheningImage, (654, 874))

#edge identification for ease of cartooning
    edgeIdentificationImage = cv2.adaptiveThreshold(smootheningImage, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY, 9, gScale)
    edgedImageSized = cv2.resize(edgeIdentificationImage, (654, 874))

#reducing color pallette for cartoon generation using machine learning
#Using k means clustering for finding the optimum color groups for cartoonising.
    data = np.float32(realImage).reshape((-1,3))
    criteria = (cv2.TERM_CRITERIA_EPS+ cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    ret , label , center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result = center[label.flatten()]
    result=result.reshape(realImage.shape)

#keeping edge sharp as required and application of bilateral filer
    bilateralFilterImage = cv2.bilateralFilter(result, 9, 250, 250)
    bilateralFilterImageSized = cv2.resize(bilateralFilterImage, (654, 874))

#cartoonification of the image by masking the edges
    finalCartoonImage = cv2.bitwise_and(bilateralFilterImage, bilateralFilterImage, mask=edgeIdentificationImage)
    cartoonifiedImageSized = cv2.resize(finalCartoonImage, (654, 874))

#transition states of images from real to cartoon
    statesOfImages=[sizeChange, greyScaledSized, smoothSized, edgedImageSized, bilateralFilterImageSized, cartoonifiedImageSized]
    fig, axes = plt.subplots(3,2, figsize=(8,8), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(statesOfImages[i], cmap='gray')
    plt.show()

#final output cartoon images
    plt.imshow(cartoonifiedImageSized)
    plt.show()

#comparison between real and cartoon image
    comparisonOfImages=[sizeChange, cartoonifiedImageSized]
    fig, axes = plt.subplots(1,2, figsize=(8,8), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(comparisonOfImages[i], cmap='gray')
    plt.show()


#UI for uploading real image
window=tk.Tk()
window.geometry('1000x1000')
window.title('Its Cartoon Time Folks !')
window.configure(background='#cce1f2')
title=Label(window,background='#f9ded7', font=('ariel',20,'bold'))
uploadButton=Button(window,text="UPLOAD IMAGE",command=uploadOriginal,padx=70,pady=20)
uploadButton.configure(background='#f9ded7', foreground='white',font=('ariel',20,'bold'))
uploadButton.pack(side=TOP,pady=300)
window.mainloop()
